In [39]:
!pip install seaborn ncps torch pytorch-lightning

In [40]:
# Load the dataset
import pandas as pd

df = pd.read_csv('../datasets/treated/HaLT-SubjectA-160223.csv')
df = df.sort_values(by=df.columns[0])
df = df.drop(df.columns[0], axis=1)
df.head()


,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,...,F8,T3,T4,T5,T6,Fz,Cz,Pz,X5,Marker
27232,-4.16,12.80,-6.16,1.66,-0.86,1.46,-1.43,-1.92,-0.02,-1.42,...,6.29,-3.82,0.99,-1.31,-2.62,1.07,2.60,-0.96,-0.11,0
22201,-6.68,14.11,-6.04,1.44,-2.30,0.87,-0.53,-0.08,0.10,-0.22,...,5.06,-4.14,1.42,-2.06,-1.17,1.68,2.26,-0.92,0.19,0
3773,-8.79,13.64,-8.61,2.38,-3.34,2.10,0.04,3.84,2.44,3.17,...,4.95,-2.66,3.54,1.12,5.75,-0.53,0.52,2.09,0.26,0
5995,-2.25,14.96,-4.80,0.86,-3.18,2.41,-3.71,0.72,-2.23,1.93,...,5.34,-3.09,2.54,-3.26,1.15,-1.01,0.08,-0.02,-0.31,0
17313,-0.09,13.47,3.72,0.12,0.00,0.06,1.20,3.26,3.48,3.11,...,0.07,-1.03,-1.43,-0.48,2.81,3.98,2.81,1.99,0.33,0


In [1]:
import torch.nn as nn
import torchmetrics
from ncps.wirings import AutoNCP
from ncps.torch import CfC
import pytorch_lightning as pl
import torch


# LightningModule for training a RNNSequence module
from typing import Any


class LiquidBlock(pl.LightningModule):
    def __init__(self, units=20, in_features=22, out_features=10, lr=0.005, weights=None):
        super().__init__()
        if weights:
            self.criterion = nn.CrossEntropyLoss(weight=weights, reduction='sum')
        else:
            self.criterion = nn.CrossEntropyLoss(reduction='sum')

        self.wiring = AutoNCP(units, out_features)
        self.cfc = CfC(in_features, self.wiring, batch_first=True)
        self.softmax = nn.Softmax(dim=1)

        self.train_acc = torchmetrics.Accuracy(task='multiclass', num_classes=out_features)
        self.valid_acc = torchmetrics.Accuracy(task='multiclass', num_classes=out_features)
        
        self.lr = lr

    def forward(self, inputs, hiddens=None) -> Any:
        x, hidden = self.cfc.forward(input=inputs, hx=hiddens)
        return self.softmax(x), hidden

    def training_step(self, batch, batch_idx, hiddens=None):
        x, y = batch
        y_hat, hidden = self.forward(x, hiddens)
        y_hat = y_hat.view_as(y)
        loss = self.criterion(y_hat, y)
                
        self.log("train_loss", loss, prog_bar=True)

        self.train_acc(y_hat, y)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False)

        return {"loss": loss, "hiddens": hidden}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat, _ = self.forward(x)
        y_hat = y_hat.view_as(y)
        loss = self.criterion(y_hat, y)

        self.log("val_loss", loss, prog_bar=True)

        self.valid_acc(y_hat, y)
        self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)

        return loss
    
    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        return torch.optim.Adam(self.cfc.parameters(), lr=self.lr)
    
    def draw_graph(self, **kwargs):
        return self.wiring.draw_graph(**kwargs)

In [17]:
test = torch.randn(64, 500, 10)
print(test.shape)
print(test.reshape(64, 1, -1).shape)
print(test.size(0))

torch.Size([64, 500, 10])
torch.Size([64, 1, 5000])
64


Encode target classes as One Hot

In [42]:
import numpy as np

# Lookup features
print(np.unique(df['Marker']))

_tensor = torch.Tensor(df['Marker'].values).int()

classes = np.unique(_tensor)
print(classes)


_tensor_one_hot = torch.nn.functional.one_hot(_tensor.long())
print(_tensor_one_hot)
print(_tensor_one_hot.shape)

[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])
torch.Size([218106, 7])


In [43]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_model(model):
    sns.set_style("white")
    plt.figure(figsize=(18, 10))
    legend_handles = model.draw_graph(draw_labels=True, neuron_colors={"command": "tab:cyan"})
    plt.legend(handles=legend_handles, loc="upper center", bbox_to_anchor=(1, 1))
    sns.despine(left=True, bottom=True)
    plt.tight_layout()
    plt.show()

Split the data for training

In [44]:
import torch.utils.data as data 

# Train the model
torch.set_float32_matmul_precision('high')

# features = df_copy[['Cz', 'C3', 'A1', 'F3', 'P3']]
features = df.loc[:, df.columns != 'Marker']

BATCH_SIZE = 50

# One hot encoding Marker
_tensor = torch.Tensor(df['Marker'].values).int()

target_one_hot = torch.nn.functional.one_hot(_tensor.long(), len(classes)).float()
# use 20% of training data for validation
train_set_size = int(len(df) * 0.8)
valid_set_size = len(df) - train_set_size

train_set = features[:][:train_set_size]
valid_set = features[:][train_set_size:]

train_data_y = target_one_hot.clone().detach()[:train_set_size]
validation_data_y = target_one_hot.clone().detach()[train_set_size:]

train_data_x = torch.tensor(train_set.values).float()
validation_data_x = torch.tensor(valid_set.values).float()

train_tensor = data.TensorDataset(train_data_x, train_data_y)
validation_tensor = data.TensorDataset(validation_data_x, validation_data_y)

train_loader = data.DataLoader(train_tensor, batch_size=BATCH_SIZE, num_workers=8)
validation_loader = data.DataLoader(validation_tensor, batch_size=BATCH_SIZE, num_workers=8)

In [48]:
# Definition of the model:
out_features = len(classes)
in_features = len(features.columns)
learn_rate = 1e-5

EPOCHS = 15

model = LiquidBlock(units=100, in_features=in_features, out_features=out_features, lr=learn_rate)
trainer = pl.Trainer(
    accelerator='gpu',
    logger=pl.loggers.CSVLogger("log"),
    max_epochs=EPOCHS,
    gradient_clip_val=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [49]:
# Train the model
trainer.fit(model, train_loader, validation_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | criterion | CrossEntropyLoss   | 0     
1 | cfc       | CfC                | 41.0 K
2 | softmax   | Softmax            | 0     
3 | train_acc | MulticlassAccuracy | 0     
4 | valid_acc | MulticlassAccuracy | 0     
-------------------------------------------------
32.9 K    Trainable params
8.1 K     Non-trainable params
41.0 K    Total params
0.164     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch 0:   0%|          | 4/3490 [00:00<05:25, 10.71it/s, v_num=65, train_loss=96.00]

/home/chema/Documents/MasterIA/TFM/code/liquid-eeg/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [47]:
def evaluate_prediction(model, df):
    with torch.no_grad():
        prediction, _ = model(torch.tensor(df.loc[:, df.columns != 'Marker'].values).float().to(device=[0]))
    class_pred = []
    for one_hot in prediction:
        idx = torch.argmax(one_hot)
        class_pred.append(idx)

    plt.plot(range(len(class_pred)), class_pred)
    plt.plot(range(len(df['Marker'])), df['Marker'])

evaluate_prediction(model, df)

TypeError: to() received an invalid combination of arguments - got (device=list, ), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)
